In [32]:
# Directory 
import os
os.chdir("/home/jovyan/work/quant_methods/Assessment/Group Work/Regression")
print(os.getcwd())

/home/jovyan/work/quant_methods/Assessment/Group Work/Regression


In [33]:
# importing libraries
import pandas as pd

In [34]:
df1 = pd.read_csv('df_per_coll_London_cyc_s1.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24594 entries, 0 to 24593
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   accident_index             24594 non-null  object 
 1   accident_year_x            24594 non-null  int64  
 2   accident_reference_x       24594 non-null  object 
 3   longitude                  24594 non-null  float64
 4   latitude                   24594 non-null  float64
 5   police_force               24594 non-null  int64  
 6   accident_severity          24594 non-null  int64  
 7   date                       24594 non-null  object 
 8   day_of_week                24594 non-null  int64  
 9   local_authority_district   24594 non-null  int64  
 10  road_type                  24594 non-null  int64  
 11  speed_limit                24594 non-null  int64  
 12  light_conditions           24594 non-null  int64  
 13  weather_conditions         24594 non-null  int

We will change our `light_conditions` because 1 represents light and all other categories represent various forms of darkness.In our regression, we will use 1 to represent light and 0 to represent darkness.

In [35]:
df1['light'] = df1['light_conditions'].apply(lambda x: 1 if x == 1 else 0)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24594 entries, 0 to 24593
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   accident_index             24594 non-null  object 
 1   accident_year_x            24594 non-null  int64  
 2   accident_reference_x       24594 non-null  object 
 3   longitude                  24594 non-null  float64
 4   latitude                   24594 non-null  float64
 5   police_force               24594 non-null  int64  
 6   accident_severity          24594 non-null  int64  
 7   date                       24594 non-null  object 
 8   day_of_week                24594 non-null  int64  
 9   local_authority_district   24594 non-null  int64  
 10  road_type                  24594 non-null  int64  
 11  speed_limit                24594 non-null  int64  
 12  light_conditions           24594 non-null  int64  
 13  weather_conditions         24594 non-null  int

In [36]:
df1 = df1.drop(columns=['light_conditions'])
df1.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24594 entries, 0 to 24593
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   accident_index             24594 non-null  object 
 1   accident_year_x            24594 non-null  int64  
 2   accident_reference_x       24594 non-null  object 
 3   longitude                  24594 non-null  float64
 4   latitude                   24594 non-null  float64
 5   police_force               24594 non-null  int64  
 6   accident_severity          24594 non-null  int64  
 7   date                       24594 non-null  object 
 8   day_of_week                24594 non-null  int64  
 9   local_authority_district   24594 non-null  int64  
 10  road_type                  24594 non-null  int64  
 11  speed_limit                24594 non-null  int64  
 12  weather_conditions         24594 non-null  int64  
 13  road_surface_conditions    24594 non-null  int

In [37]:
# Group data by location, year, severity, and additional predictors
df_aggregated = df1.groupby(
    ['accident_year_x', 'light', 'speed_limit', 'road_surface_conditions', 'weather_conditions' ]
).size().reset_index(name='accident_count')

# Check the aggregated data
print(df_aggregated.head())


   accident_year_x  light  speed_limit  road_surface_conditions  \
0             2019      0           -1                        1   
1             2019      0           20                        1   
2             2019      0           20                        1   
3             2019      0           20                        1   
4             2019      0           20                        1   

   weather_conditions  accident_count  
0                   8               1  
1                   1             233  
2                   2               3  
3                   4               2  
4                   8               4  


In [38]:
# Count the occurrences of 9 or -1 in each column
# 9 represents "unknown (self reported)"
# -1 represents "Data missing or out of range"
count_9 = (df_aggregated == 9).sum()
count_neg1 = (df_aggregated == -1).sum()
count_99 = (df_aggregated == 99).sum()

print("Count of attributes with value 9 per column:")
print(count_9)
print("\nCount of attributes with value 99 per column:")
print(count_99)
print("\nCount of attributes with value -1 per column:")
print(count_neg1)

Count of attributes with value 9 per column:
accident_year_x             0
light                       0
speed_limit                 0
road_surface_conditions    90
weather_conditions         77
accident_count             11
dtype: int64

Count of attributes with value 99 per column:
accident_year_x            0
light                      0
speed_limit                0
road_surface_conditions    0
weather_conditions         0
accident_count             0
dtype: int64

Count of attributes with value -1 per column:
accident_year_x            0
light                      0
speed_limit                4
road_surface_conditions    0
weather_conditions         0
accident_count             0
dtype: int64


In [39]:
# Filter out rows where road_surface_conditions == 9 or sex_of_casualty == -1
df_cleaned = df_aggregated[~((df_aggregated['road_surface_conditions'] == 9) | (df_aggregated['weather_conditions'] == 9) | 
                             (df_aggregated['speed_limit'] == -1))]

In [40]:
df_cleaned.to_csv('df_cleaned2.csv', index=False)